In [3]:
from functions import train_predict_ml, cross_validation, load_data
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score
from hyperopt import fmin, tpe, Trials, STATUS_OK
from hyperopt import hp
import lightgbm as lgb
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [28]:
(X_train, X_test, y_train, y_test) = load_data(data_train="../Data/train_hog.csv", data_test="../Data/test_hog.csv")

In [3]:
y_train=y_train[:400]

In [4]:
X_train=X_train[:400]

In [5]:
X_train.shape

(400, 324)

In [29]:
# space = {
#     'num_leaves':  hp.choice('num_leaves', range(50,100)),
#     'max_depth':  hp.choice('max_depth', range(3,8)),
#     'learning_rate':  hp.uniform('learning_rate', 0.001, 0.1),
#     'min_child_samples': hp.choice('min_child_samples',(5,15)),
#     'reg_alpha': hp.uniform('reg_alpha',0.05,0.2)
# }   
space = {
    'num_leaves':  hp.choice('num_leaves', range(20,100)),
    'max_depth':  hp.choice('max_depth', range(3,15)),
    'learning_rate':  hp.uniform('learning_rate', 0.001, 0.1),
    'min_child_samples': hp.choice('min_child_samples',(5,15)),
    'reg_alpha': hp.uniform('reg_alpha',0.05,0.3)
}   

In [30]:
def objective_function(params):
    clf = LGBMClassifier(**params)
    score = cross_val_score(clf, X_train, y_train, cv=4).mean()
    print(params, 'loss:', -score)
    return {'loss': -score, 'status': STATUS_OK}    
    
num_eval = 25
trials = Trials()
best_param = fmin(objective_function, space, algo=tpe.suggest, max_evals=num_eval, trials=trials, rstate=np.random.default_rng(seed=42))

{'learning_rate': 0.0655224975751405, 'max_depth': 8, 'min_child_samples': 15, 'num_leaves': 71, 'reg_alpha': 0.25341385191007193}
loss:                                                                                                                  
-0.59124                                                                                                               
{'learning_rate': 0.006933682719058652, 'max_depth': 14, 'min_child_samples': 15, 'num_leaves': 47, 'reg_alpha': 0.12510533869762708}
loss:                                                                                                                  
-0.49920000000000003                                                                                                   
{'learning_rate': 0.016560790355627814, 'max_depth': 5, 'min_child_samples': 5, 'num_leaves': 43, 'reg_alpha': 0.08674069682125694}
loss:                                                                                                                  
-0.

In [1]:
best_param={'learning_rate': 0.09767600511205632, 'max_depth': 13, 'min_child_samples': 5, 'num_leaves': 88, 'reg_alpha': 0.13123725784077744}

In [4]:
lgbm_tuned = LGBMClassifier(**best_param)
lgbm_tuned

LGBMClassifier(learning_rate=0.09767600511205632, max_depth=13,
               min_child_samples=5, num_leaves=88,
               reg_alpha=0.13123725784077744)